In [1]:
%run ../demos/qsu.ipynb
from qecsim import paulitools as pt
from pymatching import Matching
import numpy as np
from qecsim.models.rotatedplanar import RotatedPlanarCode, RotatedPlanarReweightedMWPMDecoder
from qecsim.models.generic import INIDErrorModel
import pandas as pd

In [2]:
rng = np.random.default_rng(1)
t = 3e-5
NUM_ERRORS = int(1e5)

# T_1 and T_2 data from ibm_sherbrooke 11.03.2024 
T_1 = [4.47131398e-04, 2.26458412e-04, 2.92798247e-04, 2.90948197e-04, 3.40581391e-04, 2.83836831e-04, 2.70656858e-04, 1.53601864e-04, 3.61852447e-04, 2.42139994e-04, 2.00143049e-04, 3.61774782e-04, 2.45283244e-04, 3.47100922e-04, 2.93481282e-04, 4.21930409e-04, 2.52775994e-04, 2.77442387e-04, 2.46129903e-04, 3.78459014e-04, 2.45232302e-04, 2.96744679e-04, 1.78567010e-04, 3.93160649e-04, 3.59094685e-04, 2.86956081e-04, 3.45384767e-04, 3.86961497e-04, 2.05691776e-04, 4.28754692e-04, 3.74780068e-04, 2.88966813e-04, 2.47889883e-04, 3.86880708e-04, 1.24536308e-04, 2.62453326e-04, 9.64438829e-05, 2.64377947e-04, 1.28466986e-04, 1.31691725e-04, 4.36206180e-04, 2.22960692e-04, 4.25424927e-04, 3.95216491e-04, 2.28763892e-04, 4.03873696e-04, 2.71036498e-04, 3.23739417e-04, 3.55001763e-04, 4.63425047e-05, 2.94978244e-04, 2.28271419e-04, 4.06929723e-04, 2.56146538e-04, 1.72188791e-04, 1.73462107e-04, 2.74696491e-04, 2.30326227e-04, 3.00553262e-04, 2.27812102e-04, 2.89711877e-04, 2.59015813e-04, 1.16374746e-04, 2.10523923e-04, 3.21825423e-04, 2.52531191e-04, 3.50659516e-04, 3.59079952e-04, 2.68641573e-04, 2.33773519e-04, 2.97749853e-04, 2.08399114e-04, 2.43029285e-04, 2.85902283e-04, 2.84630852e-04, 3.55945256e-04, 2.97832844e-04, 2.36402484e-04, 2.49531082e-04, 1.33130091e-04, 2.19092374e-04, 2.40286350e-04, 3.34206274e-04, 3.67597836e-04, 2.55269321e-04, 2.47502383e-04, 2.61491269e-04, 2.02710898e-04, 2.40839962e-04, 3.28335949e-04, 2.54947652e-04, 1.86721803e-04, 4.09207513e-04, 3.30273185e-04, 2.61297556e-04, 2.64197469e-04, 1.26940583e-04, 2.75063562e-04, 2.02244248e-04, 3.59510470e-04, 1.84129054e-04, 2.68350896e-04, 2.43577740e-04, 2.18748598e-04, 5.43395245e-05, 3.00174758e-04, 1.12444264e-04, 2.96240040e-04, 2.45289428e-04, 1.33545326e-04, 3.30455266e-04, 1.86542047e-04, 1.80401351e-04, 3.43144232e-04, 2.29711485e-04, 2.86391366e-04, 1.98113525e-04, 2.63177780e-04, 3.06908883e-04, 3.28242820e-04, 1.92885813e-04, 1.40990586e-04, 7.05505794e-05, 2.17445019e-04, 3.10653668e-04]    
T_2 = [2.63153399e-04, 2.86761491e-04, 1.41151669e-04, 1.66627797e-04, 1.89576638e-04, 2.92269093e-04, 2.25063209e-04, 1.81702907e-04, 2.61984119e-04, 6.11325611e-05, 1.51575773e-04, 7.41630478e-05, 2.20899934e-04, 1.02816518e-04, 5.27691939e-05, 1.76258452e-04, 7.50720939e-05, 7.53161361e-05, 5.95427485e-05, 4.67441780e-05, 6.18971486e-05, 2.34925562e-05, 2.04210901e-04, 2.20918687e-04, 1.13223403e-04, 4.50195613e-04, 2.14757890e-04, 1.48510585e-04, 7.82218006e-05, 1.92858061e-04, 2.79777668e-04, 3.21622414e-05, 1.26789703e-04, 2.57254131e-04, 1.03558923e-04, 5.48399307e-05, 4.24278864e-05, 3.44606876e-04, 6.01732031e-05, 6.37432976e-05, 2.06647237e-04, 3.28893299e-04, 3.36020790e-04, 2.55291187e-04, 2.83855122e-04, 3.11659323e-04, 3.16825279e-04, 1.20567129e-04, 3.31208982e-04, 7.45987011e-05, 1.50608995e-04, 7.74456392e-05, 2.81359658e-04, 5.08315903e-05, 2.03901246e-04, 1.69324501e-05, 6.82253376e-05, 1.20970691e-04, 2.80741944e-04, 2.62828664e-04, 2.86550495e-04, 2.31743470e-04, 8.82750115e-05, 1.96014297e-04, 2.98465911e-04, 6.06746939e-05, 2.26737507e-04, 6.31369444e-05, 1.86726857e-05, 2.47944963e-04, 1.26382882e-04, 2.54025918e-04, 1.78993791e-04, 1.91150516e-04, 1.79769739e-04, 8.40042940e-05, 1.70200659e-04, 3.51177500e-04, 2.96748335e-05, 1.39399172e-04, 1.90544238e-04, 2.02677259e-04, 4.09724041e-05, 3.11031137e-04, 1.01932181e-04, 1.07444847e-04, 3.08357110e-04, 1.29828189e-04, 5.47172067e-05, 2.86959303e-04, 1.45406519e-04, 1.04390303e-04, 2.40856220e-05, 2.90835503e-04, 1.36119607e-04, 1.35983968e-04, 1.85294934e-04, 2.28204312e-04, 5.14639823e-05, 3.78015232e-04, 7.73972055e-05, 1.87827250e-04, 2.29362850e-04, 2.91204961e-04, 4.20846188e-05, 1.85991695e-04, 1.24689673e-04, 2.96066649e-04, 1.72330249e-04, 2.51645808e-04, 4.66399761e-05, 6.16320063e-05, 2.21848271e-04, 2.79667140e-04, 3.75513828e-04, 2.10413362e-04, 2.57497350e-04, 2.07386123e-04, 3.13663945e-04, 3.62394113e-05, 3.18518580e-04, 1.79348759e-04, 1.40142464e-04, 2.65291418e-05, 8.90078292e-05]

weights_T1 = np.abs(np.log(1-np.exp(-t/np.array(T_1))))
weights_T2 = np.abs(np.log(1-np.exp(-t/np.array(T_2))))

my_code = RotatedPlanarCode(5, 5)
my_error_model = INIDErrorModel(T_1[0:my_code.n_k_d[0]], T_2[0:my_code.n_k_d[0]])
my_decoder = RotatedPlanarReweightedMWPMDecoder(my_code, t, np.array(T_1[0:my_code.n_k_d[0]]), np.array(T_2[0:my_code.n_k_d[0]]))

In [3]:
# sample errors from the error model
errors = [my_error_model.generate(my_code, t , rng) for _ in range(NUM_ERRORS)]

## Analyze Performance of Pymatching Decoder vs my Decoder

In [4]:
# Decode with QECsim
data = []
for e, error in enumerate(errors):
    syndrome = pt.bsp(error, my_code.stabilizers.T)
    recovery = my_decoder.decode(my_code, syndrome)
    recovery_str = pt.bsf_to_pauli(recovery)
    success = not (any(pt.bsp(recovery ^ error, my_code.stabilizers.T)) or any(pt.bsp(recovery ^ error, my_code.logicals.T)))  
    data.append({'error': e, 'recovery_str': recovery_str, 'success': success})
df_my_decoder = pd.DataFrame(data)
df_my_decoder.head()

Failed to load clib: libpypm.so.


,error,recovery_str,success
0,0,IXZZIIIIIIIIIIIIIIZZIIIII,True
1,1,IIIZIIIIIIYIIIIIYIIZIIIII,True
2,2,IIIIIIIIIIIIIIZIIIYZIIXII,True
3,3,IIYIIIIIIYIXIIIIIIIIIIIII,True
4,4,IIIIIIIIZIIIIIIIIIIIIZIII,True


In [5]:
# p_Z = 1/2 * (1  - np.exp(-t/np.array(T_2)))
# p_X = 1/2 * (1 - np.exp(-t/np.array(T_1)))
# weights_T1 = np.log((1-p_X)/(p_X))
# weights_T2 = np.log((1-p_Z)/(p_Z))

parity_check_matrix = my_code.stabilizers
matching = Matching(parity_check_matrix, np.concatenate((weights_T2[0:my_code.n_k_d[0]], weights_T1[0:my_code.n_k_d[0]])))

data = []
# syndromes = [pt.bsp(error, my_code.stabilizers.T) for error in errors]
syndromes = [parity_check_matrix @ np.concatenate(np.flipud(np.hsplit(error, 2))) % 2 for error in errors]
recoveries = matching.decode_batch(syndromes)
recoveries = [np.concatenate(np.flipud(np.hsplit(recovery, 2))) for recovery in recoveries]
for r, recovery in enumerate(recoveries):
    error = errors[r]
    # X and Z are swapped in the output of the pymatching algorithm
    # recovery = np.concatenate((recovery[parity_check_matrix.shape[0]+1:],recovery[0:parity_check_matrix.shape[0]+1]))
    recovery_str = pt.bsf_to_pauli(recovery)
    success = not (any(pt.bsp(recovery ^ error, my_code.stabilizers.T)) or any(pt.bsp(recovery ^ error, my_code.logicals.T)))  
    data.append({'error': r, 'recovery_str': recovery_str, 'success': success})
df_pymatching = pd.DataFrame(data)
df_pymatching.head()

,error,recovery_str,success
0,0,IXZZIIIIIIIIIIIIIIZZIIIII,True
1,1,IIIZIIIIIIYIIIIIYIIZIIIII,True
2,2,IIIIIIIIIIIIIIZIIIYZIIXII,True
3,3,IIYIIIIIIYIXIIIIIIIIIIIII,True
4,4,IIIIIIIIZIIIIIIIIIIIIZIII,True


In [6]:
# use pymatching and separately decode X and Z errors
X_checks = np.vsplit(np.hsplit(parity_check_matrix, 2)[1],2)[0] # X checks detect Z errors
Z_checks = np.vsplit(np.hsplit(parity_check_matrix, 2)[0],2)[1] # Z checks detect X errors

errors_x, errors_z = np.hsplit(np.array(errors), 2)
syndromes_x = [X_checks @ error % 2 for error in errors_x]
syndromes_z = [Z_checks @ error % 2 for error in errors_z]

matching_x = Matching(X_checks, weights_T1[0:my_code.n_k_d[0]])
matching_z = Matching(Z_checks, weights_T2[0:my_code.n_k_d[0]])

recoveries_x = matching_x.decode_batch(syndromes_x)
recoveries_z = matching_z.decode_batch(syndromes_z)

data = []
for r in range(len(recoveries_x)):
    recovery = np.concatenate((recoveries_x[r], recoveries_z[r]))
    error = errors[r]
    recovery_str = pt.bsf_to_pauli(recovery)
    success = not (any(pt.bsp(recovery ^ error, my_code.stabilizers.T)) or any(pt.bsp(recovery ^ error, my_code.logicals.T)))
    data.append({'error': r, 'recovery_str': recovery_str, 'success': success})
df_pymatching_separate = pd.DataFrame(data)
df_pymatching_separate.head()

,error,recovery_str,success
0,0,IXZZIIIIIIIIIIIIIIZZIIIII,True
1,1,IIIZIIIIIIYIIIIIYIIZIIIII,True
2,2,IIIIIIIIIIIIIIZIIIYZIIXII,True
3,3,IIYIIIIIIYIXIIIIIIIIIIIII,True
4,4,IIIIIIIIZIIIIIIIIIIIIZIII,True


In [7]:
df_my_decoder['success'].value_counts(normalize=True)

success
True     0.82523
False    0.17477
Name: proportion, dtype: float64

In [8]:
df_pymatching['success'].value_counts(normalize=True)

success
True     0.82523
False    0.17477
Name: proportion, dtype: float64

In [9]:
df_pymatching_separate['success'].value_counts(normalize=True)

success
True     0.82523
False    0.17477
Name: proportion, dtype: float64

In [11]:
df_my_decoder[df_pymatching['recovery_str'] != df_my_decoder["recovery_str"]]["success"].value_counts()

success
True     10776
False     4432
Name: count, dtype: int64

In [20]:
df_pymatching[df_pymatching_separate['recovery_str'] != df_pymatching["recovery_str"]]["success"].value_counts()

Series([], Name: count, dtype: int64)

In [12]:
df_comparison = pd.concat([df_my_decoder["recovery_str"], df_pymatching["recovery_str"], df_pymatching['recovery_str'] == df_my_decoder["recovery_str"]], axis=1, keys=['my_decoder', 'pymatching', 'is_equal'])

In [13]:
actual_observables = (errors @ logicals.T) % 2
print(actual_observables)
p_Z = 1/2 * (1  - np.exp(-t/np.array(T_2)))
p_X = 1/2 * (1 - np.exp(-t/np.array(T_1)))
weights_T1 = np.log((1-p_X)/(p_X))
weights_T2 = np.log((1-p_Z)/(p_Z))
matching = Matching(parity_check_matrix, np.concatenate((weights_T1[0:my_code.n_k_d[0]], weights_T2[0:my_code.n_k_d[0]])))
predicted_observables = matching.decode_batch(syndromes)
print(predicted_observables)
num_errors = np.sum(np.any(predicted_observables != actual_observables, axis=1))
print("numbe of errors: ", num_errors)

NameError: name 'logicals' is not defined